# Perform some data selection and grouping

In [ ]:
import sys

pwd = !echo ${PWD}
sys.path.append(pwd[0]+"/../../../code/local/bin")

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy import ndimage
import math
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.ticker import FormatStrFormatter

import seppy
import os

import ipywidgets as widgets

datapath=pwd[0]+"/../dat/"
figpath=pwd[0]+"/../fig/"

In [ ]:
# utility functions

def plotseis(d, p=0.9, valmin=1, valmax=-1, tmin=0, tmax=0.35, xmin=-800, xmax=800, grid=True, colorbar=False, curve=np.array([]), linestyle='-'):
    """Plot 2D seismic image"""
 
    vmin=np.min(d) / p
    vmax=p*np.max(d)
    
    if p<0:
        vmax=-p*np.amax(d)
        vmin=-vmax
        
    if valmin<valmax:
        vmin=valmin
        vmax=valmax
        
    nx=d.shape[0]
                
    fig=plt.figure(figsize=(6, 4),dpi=100)
    plt.imshow(np.transpose(d),interpolation='sinc',aspect="auto",extent=[xmin,xmax,tmax,tmin],cmap='Greys',vmin=vmin,vmax=vmax)
    plt.xlabel('|Offset| (m)')
    plt.ylabel('Time (s)')
    if grid==True:
        plt.grid(color='r', linestyle='-', linewidth=0.2)
    if colorbar==True:
        plt.colorbar()
    if curve.size>0:
        plt.plot(np.linspace(xmin,xmax,nx),curve,linestyle)
    plt.ylim([tmax,tmin])
    plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
    
    plt.show()

def traveltime(v=1500, tshift=0.02, xmin=-600, xmax=600, nx=1201, y=300):
    """Compute travel time curve with constant velocity and fixed time shift and offsety"""
    
    tt = np.sqrt((np.linspace(xmin,xmax,nx))**2 + y**2)/v + tshift
    return tt

In [ ]:
# read data, mask, headers, model, source functions

sep = seppy.sep()

data_axes, data = sep.read_file(datapath+"ch5_data.H")
pdata = data.reshape(data_axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5_data.HH")
pattr = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5_mute.H")
pmute = data.reshape(axes.n,order='F').T

source_axes, data = sep.read_file(datapath+"ch5_sources.H")
psrc = data.reshape(source_axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5_sources_s.H")
psrc_s = data.reshape(axes.n,order='F').T

model_axes, data = sep.read_file(datapath+"ch5_model.H")
pmodel = data.reshape(model_axes.n,order='F').T


dt=data_axes.d[0]
dx=data_axes.d[1]
nt=data_axes.n[0]
ntr=data_axes.n[1]
ns=data_axes.n[2]

sxz=np.zeros((ns,4))
sxz[:,0]=pattr[:,0,0]
sxz[:,1]=pattr[:,0,19]*1e-5
sxz[:,2]=pattr[:,0,21]*1e-5
sxz[:,3]=pattr[:,0,4]

srxz=np.zeros((ns,ntr,6))
srxz[:,:,0]=pattr[:,:,0]
srxz[:,:,1]=pattr[:,:,19]*1e-5
srxz[:,:,2]=pattr[:,:,21]*1e-5
srxz[:,:,3]=pattr[:,:,16]*1e-5
srxz[:,:,4]=pattr[:,:,18]*1e-5
srxz[:,:,5]=pattr[:,:,24]

In [ ]:
d=pdata

def plot_gather(sp, v1, v2, tshift, offsetx, offsety):
    shift0=float(tshift)
    offx = float(offsetx)
    offy = float(offsety)
    vel1 = float(v1)
    vel2 = float(v2)
    shift = math.sqrt(offx**2 + offy**2)/vel2 + shift0
    tt = traveltime(vel1, tshift=shift, xmin=-800-offx, xmax=800-offx, nx=1601, y=offy)
    plotseis(d[sp], p=0.7, valmin=-1, valmax=1, xmin=-800, xmax=800, grid=True, curve=tt, linestyle='--')

widgets.interact(plot_gather, sp=(0,99,1), v1='1900', v2=[1900,2600,4400], tshift='0.02', offsetx='0', offsety='0')

In [ ]:
extent=[model_axes.o[1],model_axes.o[1]+(model_axes.n[1]-1)*model_axes.d[1],model_axes.o[0]+(model_axes.n[0]-1)*model_axes.d[0],model_axes.o[0]]

plt.figure(figsize=(16,6))
plt.imshow(np.transpose(pmodel[0,:,:]),cmap='jet',extent=extent,vmin=2.5,vmax=6.0,interpolation='bilinear', aspect='auto')
plt.colorbar().set_label(label="km/s",size=16)
plt.scatter(srxz[:,:,3],srxz[:,:,4],s=5,c='y',marker='.')
plt.scatter(sxz[:,1],sxz[:,2],s=50,c=sxz[:,3],marker='x')
plt.xlabel(r'X (km)',fontsize=16)
plt.ylabel(r'Depth (km)',fontsize=16)

In [ ]:
# list of shots to drop for low quality
select=np.array([4,9,14,19,22,27,35,41,46,52,57,61,62,66,71,76,77,81,87,90,91,92,95,96,97])
pattr[:,:,28]=1
pattr[select[:].astype(int),:,28]=0

# keep the first or second perf in each stage
sxz1=sxz[pattr[:,0,27]==1,:]
sxz2=sxz[pattr[:,0,27]==2,:]
srxz1=srxz[pattr[:,0,27]==1,:,:]
srxz2=srxz[pattr[:,0,27]==2,:,:]

# drop the low-quality shots above; note that shots > 80 are not needed for unstimulated FWI
# and shots < 18 are not needed for stimulated FWI
sxz3=np.delete(sxz,select,axis=0)
srxz3=srxz[sxz[:,0].astype(int),:,:]

# keep the first AND second perf in each stage then drop shots < 18 and > 80
sxz4=sxz[((pattr[:,0,27]==1) + (pattr[:,0,27]==2)) * (pattr[:,0,0]>=18) * (pattr[:,0,0]<=80),:]
srxz4=srxz[((pattr[:,0,27]==1) + (pattr[:,0,27]==2)) * (pattr[:,0,0]>=18) * (pattr[:,0,0]<=80),:,:]

In [ ]:
extent=[model_axes.o[1],model_axes.o[1]+(model_axes.n[1]-1)*model_axes.d[1],model_axes.o[0]+(model_axes.n[0]-1)*model_axes.d[0],model_axes.o[0]]

plt.figure(figsize=(16,6))
plt.imshow(np.transpose(pmodel[0,:,:]),cmap='jet',extent=extent,vmin=2.5,vmax=6.0,interpolation='bilinear', aspect='auto')
plt.colorbar().set_label(label="km/s",size=16)
plt.scatter(sxz2[:,1],sxz2[:,2],s=50,c=sxz2[:,3],marker='x')
plt.xlabel(r'X (km)',fontsize=16)
plt.ylabel(r'Depth (km)',fontsize=16)

In [ ]:
# select shots and create groups

# group 1: first perf in each stage
pdata1=pdata[sxz1[:,0].astype(int),:,:]
pmute1=pmute[sxz1[:,0].astype(int),:,:]
psrc1=psrc[:,sxz1[:,0].astype(int),:]
psrc1_s=psrc_s[:,sxz1[:,0].astype(int),:]
pattr1=pattr[sxz1[:,0].astype(int),:,:]

# group 2: second perf in each stage
pdata2=pdata[sxz2[:,0].astype(int),:,:]
pmute2=pmute[sxz2[:,0].astype(int),:,:]
psrc2=psrc[:,sxz2[:,0].astype(int),:]
psrc2_s=psrc_s[:,sxz2[:,0].astype(int),:]
pattr2=pattr[sxz2[:,0].astype(int),:,:]

# group 3: drop bad quality shots
pdata3=pdata[sxz3[:,0].astype(int),:,:]
pmute3=pmute[sxz3[:,0].astype(int),:,:]
psrc3=psrc[:,sxz3[:,0].astype(int),:]
psrc3_s=psrc_s[:,sxz3[:,0].astype(int),:]
pattr3=pattr[sxz3[:,0].astype(int),:,:]

# group 4: first and second perf in each stage such that perf >=18 and <=80
pdata4=pdata[sxz4[:,0].astype(int),:,:]
pmute4=pmute[sxz4[:,0].astype(int),:,:]
psrc4=psrc[:,sxz4[:,0].astype(int),:]
psrc4_s=psrc_s[:,sxz4[:,0].astype(int),:]
pattr4=pattr[sxz4[:,0].astype(int),:,:]

In [ ]:
# save the selections (data, mask, sources, headers)
sep.write_file(datapath+"ch5_data0_group1.H", np.transpose(pdata1), ds=np.array([dt,1,1]), os=np.array([0,-800,0]), dpath=datapath)
sep.write_file(datapath+"ch5_data0_group2.H", np.transpose(pdata2), ds=np.array([dt,1,1]), os=np.array([0,-800,0]), dpath=datapath)
sep.write_file(datapath+"ch5_data0_group3.H", np.transpose(pdata3), ds=np.array([dt,1,1]), os=np.array([0,-800,0]), dpath=datapath)
sep.write_file(datapath+"ch5_data0_group4.H", np.transpose(pdata4), ds=np.array([dt,1,1]), os=np.array([0,-800,0]), dpath=datapath)

sep.write_file(datapath+"ch5_data0_group1.HH", np.transpose(pattr1), ds=np.array([1,1,1]), os=np.array([0,-800,0]), dpath=datapath)
sep.write_file(datapath+"ch5_data0_group2.HH", np.transpose(pattr2), ds=np.array([1,1,1]), os=np.array([0,-800,0]), dpath=datapath)
sep.write_file(datapath+"ch5_data0_group3.HH", np.transpose(pattr3), ds=np.array([1,1,1]), os=np.array([0,-800,0]), dpath=datapath)
sep.write_file(datapath+"ch5_data0_group4.HH", np.transpose(pattr4), ds=np.array([1,1,1]), os=np.array([0,-800,0]), dpath=datapath)

sep.write_file(datapath+"ch5_mute0_group1.H", np.transpose(pmute1), ds=np.array([dt,1,1]), os=np.array([0,-800,0]), dpath=datapath)
sep.write_file(datapath+"ch5_mute0_group2.H", np.transpose(pmute2), ds=np.array([dt,1,1]), os=np.array([0,-800,0]), dpath=datapath)
sep.write_file(datapath+"ch5_mute0_group3.H", np.transpose(pmute3), ds=np.array([dt,1,1]), os=np.array([0,-800,0]), dpath=datapath)
sep.write_file(datapath+"ch5_mute0_group4.H", np.transpose(pmute4), ds=np.array([dt,1,1]), os=np.array([0,-800,0]), dpath=datapath)

sep.write_file(datapath+"ch5_sources0_group1.H", np.transpose(psrc1), ds=np.array([dt,1,1]), os=np.array([0,0,0]), dpath=datapath)
sep.write_file(datapath+"ch5_sources0_group2.H", np.transpose(psrc2), ds=np.array([dt,1,1]), os=np.array([0,0,0]), dpath=datapath)
sep.write_file(datapath+"ch5_sources0_group3.H", np.transpose(psrc3), ds=np.array([dt,1,1]), os=np.array([0,0,0]), dpath=datapath)
sep.write_file(datapath+"ch5_sources0_group4.H", np.transpose(psrc4), ds=np.array([dt,1,1]), os=np.array([0,0,0]), dpath=datapath)

sep.write_file(datapath+"ch5_sources0_group1_s.H", np.transpose(psrc1_s), ds=np.array([dt,1,1]), os=np.array([0,0,0]), dpath=datapath)
sep.write_file(datapath+"ch5_sources0_group2_s.H", np.transpose(psrc2_s), ds=np.array([dt,1,1]), os=np.array([0,0,0]), dpath=datapath)
sep.write_file(datapath+"ch5_sources0_group3_s.H", np.transpose(psrc3_s), ds=np.array([dt,1,1]), os=np.array([0,0,0]), dpath=datapath)
sep.write_file(datapath+"ch5_sources0_group4_s.H", np.transpose(psrc4_s), ds=np.array([dt,1,1]), os=np.array([0,0,0]), dpath=datapath)